# Computing the Maximum Lyapunov Exponent via autograd

Genrating videos to better understand the dynamics via mFTLEs

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import matplotlib.pyplot as plt
import numpy as np

# Juptyer magic: For export. Makes the plots size right for the screen 
%matplotlib inline
# %config InlineBackend.figure_format = 'retina'

%config InlineBackend.figure_formats = ['svg'] 


torch.backends.cudnn.deterministic = True
seed = np.random.randint(1,200)
seed = 59
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
print(seed)
g = torch.Generator()
g.manual_seed(seed)


# Data preparation

In [ ]:
cross_entropy = False #this leads to squared loss in the training
data_noise = 0.2
batch_size = 5000
plotlim = [-3, 3]
subfolder = 'MLE_10params' #all the files generated from this notebook get saved into this subfolder

import os
if not os.path.exists(subfolder):
    os.makedirs(subfolder)


if cross_entropy == True:
    label = 'scalar'
else: label = 'vector'


from models.training import create_dataloader
dataloader, dataloader_viz = create_dataloader('moons', noise = data_noise, batch_size = batch_size, plotlim = plotlim, random_state = seed, label = label, filename = subfolder + '/trainingset')


## Model dynamics

In [ ]:
#Import of the model dynamics that describe the neural ODE
#The dynamics are based on the torchdiffeq package, that implements ODE solvers in the pytorch setting
from models.neural_odes import NeuralODEvar

#for neural ODE based networks the network width is constant. In this example the input is 2 dimensional
hidden_dim, data_dim = 2, 2 
augment_dim = 0

#T is the end time of the neural ODE evolution, num_steps are the amount of discretization steps for the ODE solver
T, num_steps = 10, 100 #
step_size = T/num_steps
param_layers = 10 #the number of distinct parameters present in the interval. they are spread equidistant over the interval [0, T]. As there are 100 num_steps, the interval is divided into 10 parts, each of length 1 with 10 num_steps per subinterval.
bound = 0.
fp = False #this recent change made things not work anymore
turnpike = False

non_linearity = 'tanh' #'relu' #
architecture = 'inside' #outside




## Training and generating level sets

In [ ]:

num_epochs = 60 #number of optimization runs in which the dataset is used for gradient decent

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
anode = NeuralODEvar(device, data_dim, hidden_dim, augment_dim=augment_dim, non_linearity=non_linearity, 
                    architecture=architecture, T=T, time_steps=num_steps, num_params = param_layers, fixed_projector=fp, cross_entropy=cross_entropy)


optimizer_anode = torch.optim.Adam(anode.parameters(), lr=1e-3) 

In [ ]:
from models.training import doublebackTrainer

trainer_anode = doublebackTrainer(anode, optimizer_anode, device, cross_entropy=cross_entropy, turnpike = turnpike,
                         bound=bound, fixed_projector=fp, verbose = True) 
trainer_anode.train(dataloader, num_epochs)

In [ ]:
from plots.plots import classification_levelsets
classification_levelsets(anode)
plt.plot(trainer_anode.histories['epoch_loss_history'])
plt.xlim(0, len(trainer_anode.histories['epoch_loss_history']) - 1)
plt.ylim(0)
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.show()

In [ ]:

#check that right hand side is autonomous
input1 = torch.tensor([1, 0], dtype=torch.float32)
times = torch.linspace(0,T,50)
times = times[:-1]

param_step = 10/10
for t in times:
   k = int(t/param_step)
   print('output', anode.flow.dynamics(t,input1),t)
   print(k)
   print('dynamics weight:', anode.flow.dynamics.fc2_time[k].weight)
   print('right hand-side', anode.flow.dynamics.forward(t, input1))

In [ ]:
import importlib
import plots.gifs
importlib.reload(plots.gifs) # Reload the module to ensure the latest changes are applied
from plots.gifs import trajectory_gif
from IPython.display import Image

#the function trajectory_gif creates the evolution trajectories of the input data through the network
#the passing time corresponds to the depth of the network

for X_viz, y_viz in dataloader_viz:
    trajectory_gif(anode, X_viz[0:50], y_viz[0:50], timesteps=num_steps, filename = subfolder + '/trajectory', axlim = 8, dpi = 100)
    break

#Display of the generated gif
traj = Image(filename=subfolder + "/trajectory.gif")
display(traj)

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from plots.plots import plot_trajectory



# Following plots are a visual check if the autonomous subintervals are where they should be
# model, inputs, targets, timesteps should be defined before calling this function
step_size = T/num_steps 

for t in torch.arange(0,0.5,0.11):
    interval = torch.tensor([0., 1.], dtype=torch.float32) + t
    print(interval)
    plot_trajectory(anode, X_viz[0:50], y_viz[0:50],stepsize = step_size, time_interval=interval, x_lim=[-3,3], y_lim = [-3,3])
    plt.show()
    


In [ ]:
time_interval = torch.tensor([7.2, 10], dtype=torch.float32)
stepsize = T/num_steps
print(interval)

start_time = time_interval[0].item()
end_time = time_interval[1].item()
num_steps_interval = int((end_time - start_time) / stepsize)
integration_time = torch.arange(start_time, end_time + stepsize/100, stepsize) #using end_time + stepsize gave a weird inconsistency between

# for t in times:
#    k = int(t/param_step)
#    print('output', anode.flow.dynamics(t,input1),t)
#    print(k)
#    print('dynamics weight:', anode.flow.dynamics.fc2_time[k].weight)
#    print('right hand-side', anode.flow.dynamics.forward(t, input1))

plot_trajectory(anode, X_viz[0:50], y_viz[0:50],stepsize = stepsize, time_interval=time_interval)


In [ ]:
from plots.plots import classification_levelsets
import os

        
footnote = f'{num_epochs = }, {param_layers = }, {num_steps = }, {data_noise = }'
        
fig_name_base = os.path.join(subfolder, 'levelsets')
classification_levelsets(anode, fig_name_base, footnote = footnote)
from IPython.display import Image
img1 = Image(filename = fig_name_base + '.png', width = 400)

display(img1)

## Lyapunov exponent computation

In [ ]:
from FTLE import LEs

#Example of how to use the LEs function to compute Lyapunov exponents
# Define inputs
input1 = torch.tensor([1, 0], dtype=torch.float32)
input2 = torch.tensor([0, 1], dtype=torch.float32)
time_interval = torch.tensor([0, T], dtype=torch.float32)


les = LEs(input1, anode, time_interval=time_interval)
print(les)


# Plot ANODE FTLEs

In [ ]:
from FTLE import LE_grid

output_max, output_min = LE_grid(anode,x_amount = 40)

In [ ]:
#testing grid plots
importlib.reload(plots.plots) # Reload the module to ensure the latest changes are applied
from plots.plots import plot_trajectory

ax_lim = 2
stepsize = T/num_steps
interval = torch.tensor([0., T], dtype=torch.float32)

traj_amount = 9
x = torch.linspace(-2,2,traj_amount)
y = torch.linspace(-2,2,traj_amount)
X, Y = torch.meshgrid(x, y)

inputs_grid = torch.stack([X,Y], dim=-1)
inputs_grid = inputs_grid.view(-1,2) #to be able to loop through all the grid values

print(f'{inputs_grid.shape = }')
labels_grid = torch.zeros(traj_amount**2,2)
print(inputs_grid.shape[0])
print(labels_grid.shape[0])

# plot_trajectory(anode, inputs_grid, labels_grid,stepsize = stepsize, time_interval=interval, alpha_line = 0.5, show = False)
# plot_trajectory(anode, X_viz[0:40], y_viz[0:40],stepsize = stepsize, time_interval=interval)
anodeimg_max = plt.imshow(np.rot90(output_max), origin='upper', extent=(-ax_lim, ax_lim, -ax_lim, ax_lim),cmap = 'viridis')
vmin_max, vmax_max = anodeimg_max.get_clim()

plt.colorbar()  # Show color scale

plt.show()

In [ ]:
anodeimg_max = plt.imshow(np.rot90(output_max), origin='upper', extent=(-2, 2, -2, 2),cmap = 'viridis')
vmin_max, vmax_max = anodeimg_max.get_clim()

plt.colorbar()  # Show color scale
plt.savefig(subfolder + '/MLE_max.png',bbox_inches='tight', dpi=300, format='png', facecolor = 'white')
plt.close()

anodeimg_min = plt.imshow(np.rot90(output_min), origin='upper', extent=(-2, 2, -2, 2),cmap = 'viridis')#, norm=CenteredNorm(vcenter=0)) # cmap='viridis')
vmin_min, vmax_min = anodeimg_min.get_clim()

plt.colorbar()  # Show color scale
plt.savefig(subfolder + '/MLE_min.png',bbox_inches='tight', dpi=300, format='png', facecolor = 'white')
plt.close()

img1 = Image(filename = fig_name_base + '.png', width = 400)
img2 = Image(filename = subfolder + '/MLE_max.png', width = 400)
img3 = Image(filename = subfolder + '/MLE_min.png', width = 400)

display(img1,img2,img3)

Evolution of the Finite Time Lyapunov Exponents over the whole interval

In [ ]:
import imageio
import os


x_amount = 40
# Generate and save plots

T= 10
t_values_step = T/param_layers #time interval has length of the time_steps and the subinterval of constant param has the same length
eps = 0.1 * step_size #this should make sure we stay inside an interval with constant parameters for the integration
t_values = np.arange(eps, T, t_values_step)
print(t_values)
for t in t_values:
    time_interval = torch.tensor([0, t_values_step], dtype=torch.float32) + t
    
    output_max, output_min = LE_grid(anode, x_amount, time_interval)
    
    # Create a figure with 2 subplots side by side
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    
    # Plot output_min on the first subplot
    im1 = axs[0].imshow(np.rot90(output_max), origin='upper', extent=(-2, 2, -2, 2), cmap='viridis')
    # im1.set_clim(vmin=vmin_max, vmax=vmax_max)
    fig.colorbar(im1, ax=axs[0], orientation='vertical')  # Show color scale
    axs[0].set_title(f'finite time LE (max) for interval = {time_interval}', fontsize=7)
    plt.sca(axs[0])
    plot_trajectory(anode, inputs_grid, labels_grid,stepsize = step_size, time_interval=time_interval, show = False)
    
    # Plot output_max on the second subplot
    im2 = axs[1].imshow(np.rot90(output_min), origin='upper', extent=(-2, 2, -2, 2), cmap='viridis')
    fig.colorbar(im2, ax=axs[1], orientation='vertical')  # Show color scale
    axs[1].set_title(f'finite time LE (min) for interval = {time_interval}', fontsize=7)
    # im2.set_clim(vmin_min,vmax_min)
    
    
    # Adjust layout to prevent overlap
    plt.tight_layout()
    
    filename = os.path.join(subfolder, f'plot_{t:.1f}.png')
    plt.savefig(filename, bbox_inches='tight', dpi=100, format='png', facecolor='white')
    plt.close()

# Create GIF
images = []
for t in t_values:
    filename = os.path.join(subfolder, f'plot_{t:.1f}.png')
    images.append(imageio.imread(filename))

imageio.mimsave(subfolder + '/finite_time_LE_per_param_subint.gif', images, fps=1)

# Clean up (optional)
for t in t_values:
    os.remove(os.path.join(subfolder, f'plot_{t:.1f}.png'))
# os.rmdir(subfolder)

LEevo_test = Image(filename=subfolder + "/finite_time_LE_per_param_subint.gif")
display(LEevo_test)

in the autonomous case any interval of same length should give the same LE. So the above gif makes sense!

In [ ]:
import imageio
import os


if not os.path.exists(subfolder):
    os.makedirs(subfolder)

T = 10
param_layers = 10
x_amount = 30
# Generate and save plots
time_step = 0.2 #T/num_steps
t_values = np.arange(1, T-time_step, time_step)
for t in t_values:
    time_interval = torch.tensor([t, T], dtype=torch.float32)
    print(time_interval)
    output_max, output_min = LE_grid(anode, x_amount, time_interval)
    
    # Create a figure with 2 subplots side by side
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    
    # Plot output_min on the first subplot
    im1 = axs[0].imshow(np.rot90(output_max), origin='upper', extent=(-2, 2, -2, 2), cmap='viridis')
    # im1.set_clim(vmin=vmin_max, vmax=vmax_max)
    fig.colorbar(im1, ax=axs[0], orientation='vertical')  # Show color scale
    axs[0].set_title(f'finite time LE (max) for interval = {time_interval}', fontsize=7)
    plt.sca(axs[0])
    # plot_trajectory(anode, inputs_grid, labels_grid,stepsize = time_step, time_interval=time_interval, show = False)
    
    # Plot output_max on the second subplot
    im2 = axs[1].imshow(np.rot90(output_min), origin='upper', extent=(-2, 2, -2, 2), cmap='viridis')
    fig.colorbar(im2, ax=axs[1], orientation='vertical')  # Show color scale
    axs[1].set_title(f'finite time LE (min) for interval = {time_interval}', fontsize=7)
    # im2.set_clim(vmin_min,vmax_min)
    
    
    # Adjust layout to prevent overlap
    plt.tight_layout()
    
    filename = os.path.join(subfolder, f'plot_{t:.1f}.png')
    plt.savefig(filename, bbox_inches='tight', dpi=100, format='png', facecolor='white')
    plt.close()

# Create GIF
images = []
for t in t_values:
    filename = os.path.join(subfolder, f'plot_{t:.1f}.png')
    images.append(imageio.imread(filename))

imageio.mimsave(subfolder + '/finite_time_LE_test.gif', images, fps=1)

# Clean up (optional)
for t in t_values:
    os.remove(os.path.join(subfolder, f'plot_{t:.1f}.png'))

LEevo_test = Image(filename=subfolder + "/finite_time_LE_test.gif")
display(LEevo_test)

In [ ]:
import imageio
import os


T = 10
param_layers = 10
dt = 1
x_amount = 20
# Generate and save plots
time_step = 0.2
t_values = np.arange(time_step, T-time_step, time_step)
for t in t_values:
    time_interval = torch.tensor([0, t], dtype=torch.float32)
    
    output_max, output_min = LE_grid(anode, x_amount, time_interval)
    
    # Create a figure with 2 subplots side by side
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    
    # Plot output_min on the first subplot
    im1 = axs[0].imshow(np.rot90(output_max), origin='upper', extent=(-2, 2, -2, 2), cmap='viridis')
    # im1.set_clim(vmin=vmin_max, vmax=vmax_max)
    fig.colorbar(im1, ax=axs[0], orientation='vertical')  # Show color scale
    axs[0].set_title(f'finite time LE (max) for interval = {time_interval}', fontsize=7)
    
    # Plot output_max on the second subplot
    im2 = axs[1].imshow(np.rot90(output_min), origin='upper', extent=(-2, 2, -2, 2), cmap='viridis')
    fig.colorbar(im2, ax=axs[1], orientation='vertical')  # Show color scale
    axs[1].set_title(f'finite time LE (min) for interval = {time_interval}', fontsize=7)
    # im2.set_clim(vmin_min,vmax_min)
    
    
    # Adjust layout to prevent overlap
    plt.tight_layout()
    
    filename = os.path.join(subfolder, f'plot_{t:.1f}.png')
    plt.savefig(filename, bbox_inches='tight', dpi=100, format='png', facecolor='white')
    plt.close()

# Create GIF
images = []
for t in t_values:
    filename = os.path.join(subfolder, f'plot_{t:.1f}.png')
    images.append(imageio.imread(filename))

imageio.mimsave('finite_time_LE_test.gif', images, fps=1)

# Clean up (optional)
for t in t_values:
    os.remove(os.path.join(subfolder, f'plot_{t:.1f}.png'))

LEevo_test = Image(filename=subfolder + "/finite_time_LE_test.gif")
display(LEevo_test)